This notebook constructs state and action objects from the log tables, and exports them as transition tables and lists in [pickle](https://docs.python.org/3/library/pickle.html) format.

In [1]:
import pickle

import pathlib as pl
import pandas as pd
import networkx as nx

from justhink_world import create_all_worlds
from justhink_world.domain.state import NetworkState, EnvironmentState
from justhink_world.agent import Human, Robot
from justhink_world.domain.action import *

### Define paths.

In [2]:
resources_dir = pl.Path('resources')
networks_dir = resources_dir.joinpath('networks')
tables_dir = pl.Path('../processed_data/log_tables')

transition_tables_pickle_file = pl.Path(
    '../processed_data/justhink_spring21_transition_tables.pickle')
transitions_pickle_file = pl.Path(
    '../processed_data/justhink_spring21_transition_lists.pickle')

### Create activity instances.
Activities are represented in 'justhink_world' as worlds.

In [3]:
worlds = create_all_worlds()

# Print information on the activities.
# The name of the activity.
# name = 'pretest-1'
name = 'collaboration-1'

# Background network connections and costs for that activity.
print(worlds[name].env.state.network.graph.edges(data=True))

[(3, 1, {'cost': 3}), (3, 0, {'cost': 3}), (3, 2, {'cost': 3}), (3, 5, {'cost': 4}), (3, 7, {'cost': 5}), (1, 9, {'cost': 3}), (1, 2, {'cost': 2}), (1, 5, {'cost': 4}), (1, 4, {'cost': 3}), (0, 2, {'cost': 5}), (0, 7, {'cost': 4}), (2, 9, {'cost': 4}), (2, 8, {'cost': 3}), (5, 4, {'cost': 2}), (5, 7, {'cost': 3}), (5, 6, {'cost': 4}), (4, 9, {'cost': 3}), (4, 6, {'cost': 2}), (7, 6, {'cost': 2}), (9, 8, {'cost': 2})]


In [4]:
# Construct an activity name map from the names in the logs
# the latest names in the justhink_world representation package.
world_renamer = {name: name for name in worlds}
world_renamer['indiv-illustrate'] = 'tutorial'
world_renamer['collab-activity'] = 'collaboration-1'
world_renamer['collab-activity-2'] = 'collaboration-2'
world_renamer['debriefing'] = 'bye'

world_renamer

{'intro': 'intro',
 'tutorial': 'tutorial',
 'pretest-1': 'pretest-1',
 'pretest-2': 'pretest-2',
 'pretest-3': 'pretest-3',
 'pretest-4': 'pretest-4',
 'pretest-5': 'pretest-5',
 'posttest-1': 'posttest-1',
 'posttest-2': 'posttest-2',
 'posttest-3': 'posttest-3',
 'posttest-4': 'posttest-4',
 'posttest-5': 'posttest-5',
 'collaboration-1': 'collaboration-1',
 'collaboration-2': 'collaboration-2',
 'bye': 'bye',
 'indiv-illustrate': 'tutorial',
 'collab-activity': 'collaboration-1',
 'collab-activity-2': 'collaboration-2',
 'debriefing': 'bye'}

### Read the state transition log files.

In [5]:
# Discover the sate transition files.
file_list = tables_dir.glob(
    'justhink21_log_*/justhink_app-state_transition.csv')

# Make a map from participant to log file path.
files = {}
for f in file_list:
    i = int(str(f.parent).split('_')[-1])
    files[i] = f

# Read the log files.
tables = {}
for participant in files:
    df = pd.read_csv(files[participant])
    tables[participant] = df

### Construct a state object from a log table row.

In [6]:
def construct_state(row, graph, state_name='state', verbose=False):
    """Construct a state object from a log table row.

    Note: The argument state_name='state' for the current state, 
    state_name='next_state' for the next state in
    the 3-tuple logs of <state, action, next_state>
    """
    # Reconstruct the list of selected edges from string.
    edge_list_text = str(row['{}.edges'.format(state_name)])
    edge_list = edge_list_text.strip('[').strip(']').split(',')
    edges = list()
    for edge_text in edge_list:
        # if the edge list is empty, no edges are selected.
        if edge_list[0] == '':
            break
        l = edge_text.split()
        edge = (int(l[1]), int(l[3]))
        edges.append(edge)
    # Represent the selected edges as a subgraph.
    subgraph = nx.Graph()
    for u, v in edges:
        subgraph.add_edge(u, v)

    # Construct the suggested edge.
    suggested_edge = (int(row['{}.suggested.u'.format(state_name)]),
                      int(row['{}.suggested.v'.format(state_name)]))
    if suggested_edge == (-1, -1):
        suggested_edge = None

    # Construct the terminal/not and submitting/not information.
    is_terminal = bool(row['{}.terminal'.format(state_name)])
    is_submitting = bool(row['{}.submit_suggested'.format(state_name)])

    # Construct the set of active agents (indicating the turn at that state).
    if row['turn_agent'] == 'human':
        agents = frozenset({Human})
    elif row['turn_agent'] == 'robot':
        agents = frozenset({Robot})
    else:
        agents = frozenset()

    # Create the network state object.
    network = NetworkState(graph, subgraph, suggested_edge)

    # Create the environment state object.
    state = EnvironmentState(
        network, agents=agents, is_submitting=is_submitting, is_terminal=is_terminal)

    if verbose:
        print(state)

    return state


# Example / try out:
world = worlds['pretest-1']
df = tables[1].copy()
row_index = 10
construct_state(df.iloc[row_index],
                world.env.state.network.graph, verbose=True)

EnvironmentState(NetworkState(e:6+0,c:19|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0)


EnvironmentState(NetworkState(e:6+0,c:19|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0)

### Construct an action object from a log table row.

In [7]:
def construct_action(row, graph, verbose=False):
    """Construct an action object from a log table row."""
    action_type = row['action.type']
    if row['action.agent_name'] == 'human':
        agent = Human
    elif row['action.agent_name'] == 'robot':
        agent = Robot
    else:
        raise NotImplementedError

    if action_type == 0:
        return SuggestPickAction((row['action.edge.u'], row['action.edge.v']), agent)
    if action_type == 1:
        return PickAction((row['action.edge.u'], row['action.edge.v']), agent)
    if action_type == 2:
        return UnpickAction((row['action.edge.u'], row['action.edge.v']), agent)
    if action_type == 3:
        return SubmitAction(agent)
    if action_type == 4:
        return AttemptSubmitAction(agent)
    if action_type == 5:
        return ContinueAction(agent)
    if action_type == 6:
        return AgreeAction(agent)
    if action_type == 7:
        return DisagreeAction(agent)
    if action_type == 8:
        return ClearAction()
    if action_type == 9:
        print("TYPE_GUESS")
    return None


# Example / try out:
world = worlds['pretest-1']
df = tables[1].copy()
row_index = 10
construct_action(
    df.iloc[row_index], world.env.state.network.graph, verbose=True)

Action(pick(5,6),Human)

### Construct all states and actions for a log table.

Iterate through the rows.
Also append world_state, action, is_submission, cost, and is_mst columns to the table.

In [8]:
def construct_state_from_table(df, state_name='state', verbose=False):
    """Construct all states and action for a log table.

    Note: The argument state_name='state' for the current state, 
    state_name='next_state' for the next state in
    the 3-tuple logs of <state, action, next_state>
    """
    state_list = []
    is_submission = []
    cost = []
    is_mst = []
    action_list = []

    attempt_nos = dict()
    for i, row in df.iterrows():
        problem_name = row['header.frame_id']
        # Renamed scenes.
        problem_name = world_renamer[problem_name]
        problem = worlds[problem_name]

        if problem_name not in attempt_nos:
            attempt_nos[problem_name] = 1

        action = construct_action(row, problem.env.state.network.graph)
        action_list.append(action)

        state = construct_state(
            row, problem.env.state.network.graph, state_name=state_name)

        submission = identify_submission_state(row)
        is_submission.append(submission)
        if verbose and submission:
            print('Submitted', problem_name, state)

        # Update the attempt number.
        if submission:
            attempt_nos[problem_name] = attempt_nos[problem_name] + 1
        state.attempt_no = attempt_nos[problem_name]

        # Update the max attempts.
        if 'collab-activity' in problem_name:
            state.max_attempts = 4

        state_list.append(state)

        cost.append(state.network.get_cost())
        is_mst.append(state.network.is_mst())

    # append worldstate column
    df[state_name] = state_list
    # append action object column
    df['action'] = action_list
    # append is submission column
    df['is_submission'] = is_submission
    # append cost column
    df['cost'] = cost
    # append is_mst column
    df['is_mst'] = is_mst

    return df

### Identify a submission action and the associated submitted state.

In [9]:
def identify_submission_state(row):
    """Identify which problem state has been submitted"""
    # check if the last state's action type is submit type 3 or 4
    problem_name = row['header.frame_id']

    # Renamed activities.
    problem_name = world_renamer[problem_name]

    action_type = row['action.type']

    if 'test' in problem_name:
        return action_type == 4

    # For collaborative activities check for 3 only.
    elif 'collab' in problem_name:
        return action_type == 3

    else:
        return False
    

# Example / try out:
# name = 'collab-activity'
name = 'pretest-1'
df = tables[1].copy()
df = construct_state_from_table(df, state_name='next_state')
df = construct_state_from_table(df, verbose=True)

pd.options.display.max_columns = None
df

Submitted pretest-1 EnvironmentState(NetworkState(e:7+0,c:23|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0)
Submitted pretest-1 EnvironmentState(NetworkState(e:7+0,c:23|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:1,h:0)
Submitted pretest-2 EnvironmentState(NetworkState(e:7+0,c:38|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0)
Submitted pretest-2 EnvironmentState(NetworkState(e:7+0,c:38|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:1,h:0)
Submitted pretest-3 EnvironmentState(NetworkState(e:6+0,c:22|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0)
Submitted pretest-3 EnvironmentState(NetworkState(e:6+0,c:22|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:1,h:0)
Submitted pretest-4 EnvironmentState(NetworkState(e:6+0,c:45|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0)
Submitted pretest-4 EnvironmentState(NetworkState(e:6+0,c:45|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:1,h:0)
Submitted pretest-5 EnvironmentState(NetworkState(e:6+0,c:15|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0)
Submitted pretest-5 EnvironmentState(NetworkState(e:6+0,c:15|n:7,e:12;s:1)@1/inf,a

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,state.edges,state.suggested.u,state.suggested.v,state.terminal,state.submit_suggested,action.agent_name,action.type,action.edge.u,action.edge.v,next_state.edges,next_state.suggested.u,next_state.suggested.v,next_state.terminal,next_state.submit_suggested,action_no,step_no,turn_agent,next_state,action,is_submission,cost,is_mst,state
0,1.623049e+09,1,1623048864,591364622,indiv-illustrate,[],-1,-1,False,False,human,1,1,9,[u: 1\nv: 9],-1,-1,False,False,0,1,human,"EnvironmentState(NetworkState(e:1+0,c:3|n:2,e:...","Action(pick(1,9),Human)",False,0,False,"EnvironmentState(NetworkState(e:0+0,c:0|n:2,e:..."
1,1.623049e+09,2,1623048901,478822946,indiv-illustrate,[u: 1\nv: 9],-1,-1,False,False,human,8,-1,-1,[],-1,-1,False,False,1,2,human,"EnvironmentState(NetworkState(e:0+0,c:0|n:2,e:...","Action(clear,Human)",False,3,True,"EnvironmentState(NetworkState(e:1+0,c:3|n:2,e:..."
2,1.623049e+09,3,1623048918,748908519,indiv-illustrate,[],-1,-1,False,False,human,1,1,9,[u: 1\nv: 9],-1,-1,False,False,2,3,human,"EnvironmentState(NetworkState(e:1+0,c:3|n:2,e:...","Action(pick(1,9),Human)",False,0,False,"EnvironmentState(NetworkState(e:0+0,c:0|n:2,e:..."
3,1.623049e+09,4,1623048920,975822925,indiv-illustrate,[u: 1\nv: 9],-1,-1,False,False,human,3,-1,-1,[u: 1\nv: 9],-1,-1,True,False,3,4,human,"EnvironmentState(NetworkState(e:1+0,c:3|n:2,e:...","Action(submit,Human)",False,3,True,"EnvironmentState(NetworkState(e:1+0,c:3|n:2,e:..."
4,1.623049e+09,5,1623048961,5374193,pretest-1,[],-1,-1,False,False,human,1,7,0,[u: 7\nv: 0],-1,-1,False,False,0,0,human,"EnvironmentState(NetworkState(e:1+0,c:4|n:7,e:...","Action(pick(7,0),Human)",False,0,False,"EnvironmentState(NetworkState(e:0+0,c:0|n:7,e:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,1.623052e+09,282,1623051543,379220008,posttest-5,"[u: 5\nv: 4, u: 0\nv: 3, u: 6\nv: 4]",-1,-1,False,False,human,1,1,4,"[u: 5\nv: 4, u: 0\nv: 3, u: 6\nv: 4, u: 1\nv: 4]",-1,-1,False,False,3,0,human,"EnvironmentState(NetworkState(e:4+0,c:10|n:7,e...","Action(pick(1,4),Human)",False,7,False,"EnvironmentState(NetworkState(e:3+0,c:7|n:7,e:..."
282,1.623052e+09,283,1623051551,892409086,posttest-5,"[u: 5\nv: 4, u: 0\nv: 3, u: 6\nv: 4, u: 1\nv: 4]",-1,-1,False,False,human,1,7,0,"[u: 7\nv: 0, u: 5\nv: 4, u: 0\nv: 3, u: 6\nv: ...",-1,-1,False,False,4,0,human,"EnvironmentState(NetworkState(e:5+0,c:14|n:7,e...","Action(pick(7,0),Human)",False,10,False,"EnvironmentState(NetworkState(e:4+0,c:10|n:7,e..."
283,1.623052e+09,284,1623051554,113395452,posttest-5,"[u: 7\nv: 0, u: 5\nv: 4, u: 0\nv: 3, u: 6\nv: ...",-1,-1,False,False,human,1,7,6,"[u: 7\nv: 6, u: 0\nv: 3, u: 5\nv: 4, u: 7\nv: ...",-1,-1,False,False,5,0,human,"EnvironmentState(NetworkState(e:6+0,c:16|n:7,e...","Action(pick(7,6),Human)",False,14,False,"EnvironmentState(NetworkState(e:5+0,c:14|n:7,e..."
284,1.623052e+09,285,1623051556,168270349,posttest-5,"[u: 7\nv: 6, u: 0\nv: 3, u: 5\nv: 4, u: 7\nv: ...",-1,-1,False,False,human,4,-1,-1,"[u: 5\nv: 4, u: 7\nv: 0, u: 0\nv: 3, u: 6\nv: ...",-1,-1,False,True,6,0,human,"EnvironmentState(NetworkState(e:6+0,c:16|n:7,e...","Action(attempt-submit,Human)",True,16,False,"EnvironmentState(NetworkState(e:6+0,c:16|n:7,e..."


### Construct a transition table from the log table for each participant.

In [10]:
transition_tables = {}
for participant in sorted(tables):
    df = tables[participant].copy()
    
    df = construct_state_from_table(df, state_name='next_state')
    df = construct_state_from_table(df, state_name='state')

    transition_tables[participant] = df
    print('Processed', participant)

Processed 1
Processed 2
Processed 3
Processed 4
Processed 5
Processed 6
Processed 7
Processed 8
Processed 9
Processed 10


### Export the transition tables in CSV data format.

In [11]:
with transition_tables_pickle_file.open('wb') as handle:
    pickle.dump(transition_tables, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('Saved transition tables to {}'.format(transition_tables_pickle_file))

Saved transition tables to ../processed_data/justhink_spring21_transition_tables.pickle


### Construct state transition lists per activity.

In [12]:
# List of transitions, indexed by participant and then the activity name.
transition_lists = dict()

for participant, log_df in transition_tables.items():
    print('Processing participant {}:'.format(participant))
    transition_lists[participant] = dict()

    for name in world_renamer:

        df = log_df[log_df['header.frame_id'] == name].reset_index()

        trans_list = []
        for i, row in df.iterrows():
            if i == 0:
                trans_list.append(row['state'])
            trans_list.append(row['action'])
            trans_list.append(row['next_state'])

        if len(trans_list) != 0:
            print('  Added {} states for {} at {}'.format(
                len(trans_list)+1//2, participant, name))
            transition_lists[participant][world_renamer[name]] = trans_list

Processing participant 1:
  Added 19 states for 1 at pretest-1
  Added 19 states for 1 at pretest-2
  Added 19 states for 1 at pretest-3
  Added 17 states for 1 at pretest-4
  Added 17 states for 1 at pretest-5
  Added 17 states for 1 at posttest-1
  Added 17 states for 1 at posttest-2
  Added 17 states for 1 at posttest-3
  Added 17 states for 1 at posttest-4
  Added 17 states for 1 at posttest-5
  Added 9 states for 1 at indiv-illustrate
  Added 219 states for 1 at collab-activity
  Added 181 states for 1 at collab-activity-2
Processing participant 2:
  Added 7 states for 2 at pretest-1
  Added 39 states for 2 at pretest-2
  Added 19 states for 2 at pretest-3
  Added 37 states for 2 at pretest-4
  Added 21 states for 2 at pretest-5
  Added 21 states for 2 at posttest-1
  Added 17 states for 2 at posttest-2
  Added 17 states for 2 at posttest-3
  Added 17 states for 2 at posttest-4
  Added 19 states for 2 at posttest-5
  Added 9 states for 2 at indiv-illustrate
  Added 111 states for 

### Export state transition lists.

In [13]:
with transitions_pickle_file.open('wb') as handle:
    pickle.dump(transition_lists, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('Saved transition lists to {}'.format(transitions_pickle_file))

Saved transition lists to ../processed_data/justhink_spring21_transition_lists.pickle
